In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_excel('penalty_stats_updated.xlsx', sheet_name='2019')

In [8]:
df.head()

,Season,Date,Home Team,Away Team,Home Team Penality Conceded Count,Away Team Penality Conceded Count,Match Referee,Unnamed: 7
0,2019,2019-02-15 08:35:00,Chiefs,Highlanders,10,7,Glen Jackson,NZ
1,2019,2019-02-15 10:45:00,Brumbies,Rebels,6,11,Ben O'Keeffe,NZ
2,2019,2019-02-16 08:35:00,Blues,Crusaders,9,11,Nick Briant,NZ
3,2019,2019-02-16 10:45:00,Waratahs,Hurricanes,14,12,Angus Gardner,AUS
4,2019,2019-02-16 12:55:00,Sunwolves,Sharks,16,7,Nic Berry,AUS


In [9]:
def clean_penalties(value):
    
    try:
        return int(value.split('=')[1])
    except AttributeError:
        return int(value)

In [10]:
df['Home Team Penality Conceded Count'] = df['Home Team Penality Conceded Count'].apply(clean_penalties)
df['Away Team Penality Conceded Count'] = df['Away Team Penality Conceded Count'].apply(clean_penalties)

In [11]:
df['Home Team Penality Conceded Count'].sum()

821

In [12]:
df['Away Team Penality Conceded Count'].sum()

830

In [13]:
# In 2017, away teams received 73 (5.75%) more penalties than home teams. In 2018, the difference was 29 (2.36%).
# Although the competition is only -- finished in games played, the difference between home and away is nine (1.1%).

In [14]:
df.columns = ['Season', 'Date ', 'Home Team', 'Away Team',
       'Home Team Penality Conceded Count',
       'Away Team Penality Conceded Count', 'Match Referee', 'Referee Nationality']

In [15]:
df[df['Home Team'] == 'Rebels']

,Season,Date,Home Team,Away Team,Home Team Penality Conceded Count,Away Team Penality Conceded Count,Match Referee,Referee Nationality
15,2019,2019-03-01 10:45:00,Rebels,Highlanders,10,8,AJ Jacobs,RSA
22,2019,2019-03-08 10:45:00,Rebels,Brumbies,11,12,AJ Jacobs,RSA
51,2019,2019-04-06 10:45:00,Rebels,Sunwolves,16,6,Nic Berry,AUS
54,2019,2019-04-12 11:45:00,Rebels,Stormers,7,8,Mike Fraser,NZ
79,2019,2019-05-10 11:45:00,Rebels,Reds,10,14,Angus Gardner,AUS
86,2019,2019-05-17 11:45:00,Rebels,Bulls,7,11,Federico Anselmi,ARG


In [53]:
df.sort_values('Home Team Penality Conceded Count', ascending=False)['Home Team Penality Conceded Count'].mean()

9.021978021978022

In [57]:
df['Away Team Penality Conceded Count'].sum() - df['Home Team Penality Conceded Count'].sum()

9

In [18]:
df['Penalty Diff'] = df['Home Team Penality Conceded Count'] - df['Away Team Penality Conceded Count']

In [73]:
df['Label'] = df['Home Team'] + ' - ' + df['Away Team']

In [75]:
df.to_csv('penalty_diff.csv')

In [19]:
def remove_neg(value):
    if value < 0:
        value = value * -1
        return value
    else:
        return value

In [66]:
df['Penalty Diff'] = df['Penalty Diff'].apply(remove_neg)

In [69]:
df.sort_values('Penalty Diff', ascending=False).to_csv('penalty_diff.csv')

In [21]:
df[['Away Team', 'Penalty Diff']]['Penalty Diff'].mean()

3.989010989010989

In [72]:
df.to_csv('penalty_diff.csv')

In [62]:
df['Penalty Diff'].groupby(df['Referee Nationality']).mean()

Referee Nationality
ARG    3.400000
AUS    4.210526
NZ     3.871795
RSA    4.107143
Name: Penalty Diff, dtype: float64

In [23]:
def get_referee_penalty(country):
    
    # NZ, AUS, RSA, ARG, JAP

    new_df = df[df['Referee Nationality'] == country]

    away_df = new_df['Away Team Penality Conceded Count'].groupby(new_df['Away Team']).mean().reset_index()
    home_df = new_df['Home Team Penality Conceded Count'].groupby(new_df['Home Team']).mean().reset_index()

    away_df.columns = ['Team', 'Away Team Penality Conceded Count']
    home_df.columns = ['Team', 'Home Team Penality Conceded Count']

    new_df = pd.merge(away_df, home_df, on='Team', how='outer')
    
    new_df['Team Average'] = new_df[['Away Team Penality Conceded Count',
                                     'Home Team Penality Conceded Count']].mean(axis=1)
    
    new_df = new_df[['Team', 'Team Average']]
    
    new_df.columns = ['Team', '%s Ref Mean' % country]
    
    return new_df

In [24]:
ref_df = pd.merge(get_referee_penalty('RSA'), get_referee_penalty('NZ'), how='outer')
ref_df = pd.merge(ref_df, get_referee_penalty('AUS'), how='outer')
ref_df = pd.merge(ref_df, get_referee_penalty('ARG'), how='outer')
ref_df = pd.merge(ref_df, get_referee_penalty('JAP'), how='outer')

In [25]:
ref_df.index = ref_df['Team']
ref_df.drop('Team', axis=1, inplace=True)

In [26]:
ref_df['Mean'] = ref_df.mean(axis=1)

In [27]:
ref_df.sort_values('Mean', ascending=False)

,RSA Ref Mean,NZ Ref Mean,AUS Ref Mean,ARG Ref Mean,JAP Ref Mean,Mean
Team,,,,,,
Crusaders,8.833333,12.750000,13.000000,NaN,NaN,11.527778
Sunwolves,8.000000,10.000000,11.750000,14.0,NaN,10.937500
Jaguares,10.750000,11.625000,9.000000,NaN,NaN,10.458333
Bulls,9.200000,9.416667,NaN,11.0,NaN,9.872222
Rebels,11.583333,9.000000,10.250000,7.0,NaN,9.458333
Brumbies,11.000000,7.916667,9.000000,NaN,NaN,9.305556
Hurricanes,6.000000,10.125000,11.000000,NaN,NaN,9.041667
Reds,8.333333,9.250000,10.000000,8.0,NaN,8.895833
Blues,8.000000,7.950000,11.000000,7.0,NaN,8.487500


In [28]:
ref_df['RSA Ref Mean'].mean()

8.371666666666666

In [29]:
ref_df['NZ Ref Mean'].mean()

9.16888888888889

In [30]:
ref_df['AUS Ref Mean'].mean()

9.726190476190478

In [31]:
ref_df['Max - Min difference'] = ref_df.max(axis=1) - ref_df.min(axis=1)

In [32]:
ref_df.sort_values('Max - Min difference', ascending=False)

,RSA Ref Mean,NZ Ref Mean,AUS Ref Mean,ARG Ref Mean,JAP Ref Mean,Mean,Max - Min difference
Team,,,,,,,
Highlanders,9.000000,6.666667,12.000000,6.0,NaN,8.416667,6.000000
Sunwolves,8.000000,10.000000,11.750000,14.0,NaN,10.937500,6.000000
Hurricanes,6.000000,10.125000,11.000000,NaN,NaN,9.041667,5.000000
Rebels,11.583333,9.000000,10.250000,7.0,NaN,9.458333,4.583333
Chiefs,4.000000,7.666667,8.500000,4.0,NaN,6.041667,4.500000
Lions,7.625000,10.250000,6.000000,NaN,NaN,7.958333,4.250000
Crusaders,8.833333,12.750000,13.000000,NaN,NaN,11.527778,4.166667
Blues,8.000000,7.950000,11.000000,7.0,NaN,8.487500,4.000000
Waratahs,6.500000,7.750000,9.666667,7.0,NaN,7.729167,3.166667


In [33]:
def get_nationality(value):
    
    if value == 'Blues':
        national = 'NZ'
    elif value == 'Brumbies':
        national = 'AUS'
    elif value == 'Bulls':
        national = 'RSA'
    elif value == 'Cheetahs':
        national = 'RSA'
    elif value == 'Chiefs':
        national = 'NZ'
    elif value == 'Crusaders':
        national = 'NZ'
    elif value == 'Force':
        national = 'AUS'
    elif value == 'Highlanders':
        national = 'NZ'
    elif value == 'Hurricanes':
        national = 'NZ'
    elif value == 'Jaguares':
        national = 'ARG'
    elif value == 'Kings':
        national = 'RSA'
    elif value == 'Lions':
        national = 'RSA'
    elif value == 'Rebels':
        national = 'AUS'
    elif value == 'Reds':
        national = 'AUS'
    elif value == 'Sharks':
        national = 'RSA'
    elif value == 'Stormers':
        national = 'RSA'
    elif value == 'Sunwolves':
        national = 'JAP'
    elif value == 'Waratahs':
        national = 'AUS'
    
    return national

In [34]:
ref_df['Team Nationality'] = list(ref_df.reset_index()['Team'].apply(get_nationality).reset_index()['Team'])

In [37]:
ref_df

,RSA Ref Mean,NZ Ref Mean,AUS Ref Mean,ARG Ref Mean,JAP Ref Mean,Mean,Max - Min difference,Team Nationality
Team,,,,,,,,
Blues,8.000000,7.950000,11.000000,7.0,NaN,8.487500,4.000000,NZ
Brumbies,11.000000,7.916667,9.000000,NaN,NaN,9.305556,3.083333,AUS
Bulls,9.200000,9.416667,NaN,11.0,NaN,9.872222,1.800000,RSA
Chiefs,4.000000,7.666667,8.500000,4.0,NaN,6.041667,4.500000,NZ
Crusaders,8.833333,12.750000,13.000000,NaN,NaN,11.527778,4.166667,NZ
Highlanders,9.000000,6.666667,12.000000,6.0,NaN,8.416667,6.000000,NZ
Jaguares,10.750000,11.625000,9.000000,NaN,NaN,10.458333,2.625000,ARG
Lions,7.625000,10.250000,6.000000,NaN,NaN,7.958333,4.250000,RSA
Rebels,11.583333,9.000000,10.250000,7.0,NaN,9.458333,4.583333,AUS


In [47]:
ref_df[ref_df['Team Nationality'] == 'NZ']['AUS Ref Mean'].mean()

11.1

In [48]:
ref_df[ref_df['Team Nationality'] == 'AUS']['AUS Ref Mean'].mean()

9.729166666666666

In [49]:
ref_df[ref_df['Team Nationality'] == 'RSA']['AUS Ref Mean'].mean()

7.0

In [36]:
# In 2017, South African referees penalised South African teams 9.03 times per match
# During this season South African referees penalised New Zeland teams 10.69 times per match and Australians 9.2 times per match.

# New Zealand referees penalised South African teams 9.72 times per match, compared to New Zealand teams
# who were penalised 8.24 times and Australian teams who were penalised 9.18 times.

# Austrlian refs penalised South African teams 8.6 times, compared to New Zealand (9.82) and Australian teams (9.21).